본 서적의 깃허브 및 블로그 주소 </br>
https://bit.ly/transformer-git </br>
https://bit.ly/transformer-home


In [4]:
# Colab에는 트랜스포머가 설치되어 있지 않으므로 이 코드를 사용해 수동 설치
!pip install transformers

In [5]:
# 트랜스포머에서는 pipeline() 함수를 호출하면서 관심작업 이름을 전달해 파이프라인 객체를 만든다.
from transformers import pipeline

classifier = pipeline("text-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


처음 위 코드를 실행하면 파이프라인이 자동으로 허깅페이스 허브에서 모델 가중치를 다운로드 한다.  
허깅 페이스 허브 주소  
https://oreil.ly/zLK11

</br>

파라미터인 text-classification은 감성 분석을 위한 모델을 사용하지만  
다중분류와 다중 레이블 분류도 지원한다.  
  
이제 파이프라인이 있으니 예측을 만들어보자

In [7]:
text = """Dear Amazon, last week I ordered an Optimus Prime action figure \
from your online store in Germany. Unfortunately, when I opened the package, \
I discovered to my horror that I had been sent an action figure of Megatron \
instead! As a lifelong enemy of the Decepticons, I hope you can understand my \
dilemma. To resolve the issue, I demand an exchange of Megatron for the \
Optimus Prime figure I ordered. Enclosed are copies of my records concerning \
this purchase. I expect to hear from you soon. Sincerely, Bumblebee."""

우선 위와 같은 텍스트를 준비했다.  
이것은 온라인 주문에 대한 가짜 피드백으로 법률 계약, 제품 설명 등의 내용이 들어가 있다.  
이것은 과연 긍정적인 피드백인가 부정적인 피드백인가?  
</br>
한 번 분류해보자

# 텍스트 분류

In [8]:
import pandas as pd

outputs = classifier(text)
pd.DataFrame(outputs)

,label,score
0,NEGATIVE,0.901546


각 예측은 하나의 파이썬 딕셔너리 이므로 판다스를 사용한다.  
DataFrame 메소드를 통해 결과를 출력해본 결과 부정적이라는 결과가 나온다.  

</br>

자, 이걸 분석해보자

# 개체명 인식

In [10]:
ner_tagger = pipeline("ner", aggregation_strategy="simple")
outputs = ner_tagger(text)
pd.DataFrame(outputs)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


,entity_group,score,word,start,end
0,ORG,0.879010,Amazon,5,11
1,MISC,0.990859,Optimus Prime,36,49
2,LOC,0.999755,Germany,90,97
3,MISC,0.556571,Mega,208,212
4,PER,0.590255,##tron,212,216
5,ORG,0.669692,Decept,253,259
6,MISC,0.498349,##icons,259,264
7,MISC,0.775362,Megatron,350,358
8,MISC,0.987854,Optimus Prime,367,380
9,PER,0.812096,Bumblebee,502,511


이 파이프라인은 모든 개체명을 감지하고 ORG(조직), LOC(위치), PER(사람)  
같은 카테고리에 할당했다.  
  
Optimus Prime은 두 단어지만 하나의 카테고리에 할당 (MISC(그 외))에 할당했다.  
</br>

점수는 모델이 개체명을 얼마나 확신하는지 나타낸다.  
참고로 word의 ##은 토큰화로, 이에 대해선 추후에 알아보자

</br>
이번엔 질답을 해보자

# 질문 답변

In [11]:
reader = pipeline("question-answering")
question = "What does the customer want?"   # 질문
outputs = reader(question = question, context = text)
pd.DataFrame([outputs])

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


,score,start,end,answer
0,0.631292,335,358,an exchange of Megatron


질문에 대해서는 딱히 긍정도 부정도 아닌 값이 나왔다.  
</br>

이번엔 장황하게 늘어놓은 글을 요약하는 요약모델에 대해 알아보자

# 요약

In [13]:
summarizer = pipeline("summarization")
outputs = summarizer(text, max_length=60, clean_up_tokenization_spaces=True)
print(outputs[0]['summary_text'])

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


 Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I hope you can understand


다음과 같이 요약되었다.  
Bumblebee ordered an Optimus Prime action figure from your online store in Germany. Unfortunately, when I opened the package, I discovered to my horror that I had been sent an action figure of Megatron instead. As a lifelong enemy of the Decepticons, I hope you can understand  

</br>
요약이 완벽하지는 않지만 불만을 제기한 사람이 Bumblebee라는 것은  
확실하게 구분했다.  
  
</br>
이번엔 번역에 대해 알아보자

# 번역

In [22]:
# 먼저 번역에 필요한 패키지 설치
!pip install sentencepiece

In [28]:
!pip install transformers[sentencepiece]

In [31]:
from tranformers import sentencepiece
translator = pipeline("translation_en_to_de", model="Helsinki-NLP/opus-mt-en-de")
outputs = translator(text, clean_up_tokenization_spaces=True, min_length=100)
print(outputs[0]['translation_text'])

ModuleNotFoundError: ignored

독일어로 번역하는 기능이다.